# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.


### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** Remember that any API usage beyond the $200 credit will be charged to your personal account. You can set quotas and limits to your daily requests to be sure you can't be charged. Check out [Google Maps Platform Billing](https://developers.google.com/maps/billing/gmp-billing#monitor-and-restrict-consumption) and [Manage your cost of use](https://developers.google.com/maps/documentation/javascript/usage-and-billing#set-caps) for more information.

* **Note:** if you having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.

* Create a heat map that displays the humidity for every city from the part I of the homework.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import ipywidgets as widgets

# Import API key
from api_keys import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'Data', 'City_Weather.csv')
city_df = pd.read_csv(csv_file)
len(city_df)

249

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

#Use the Lat and Lng as locations and Humidity as the weight. Add Heatmap layer to map.
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions.
# A max temperature lower than 80 degrees F but higher than 70 F.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

pref_city_df = city_df.loc[(city_df["Wind Speed"] < 10) & (city_df["Max Temp"] > 70) & (city_df["Max Temp"] < 80) & (city_df["Cloudiness"] == 0), :]

#Drop any rows will null values.
pref_city_df = pref_city_df.dropna(how='any')
pref_city_df.reset_index(inplace=True)
del pref_city_df['index']
pref_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,160,Sirte,31.2089,16.5887,75.27,59,0,4.38,LY,1631772701
1,236,Shirvan,39.9509,48.8994,78.71,43,0,0.56,AZ,1631772779
2,241,Artyom,40.4723,50.3330,78.94,57,0,1.72,AZ,1631772783
3,246,Saint-Pierre,-21.3393,55.4781,78.44,64,0,5.75,RE,1631772697


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.

hotel_df=pref_city_df
pref_city_df['Hotel Name'] = ""
pref_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,160,Sirte,31.2089,16.5887,75.27,59,0,4.38,LY,1631772701,
1,236,Shirvan,39.9509,48.8994,78.71,43,0,0.56,AZ,1631772779,
2,241,Artyom,40.4723,50.3330,78.94,57,0,1.72,AZ,1631772783,
3,246,Saint-Pierre,-21.3393,55.4781,78.44,64,0,5.75,RE,1631772697,


In [7]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

for index, row in hotel_df.iterrows():
    try:      
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"
        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
#pprint(hotel_data,indent=2)
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,160,Sirte,31.2089,16.5887,75.27,59,0,4.38,LY,1631772701,Sea Hotel
1,236,Shirvan,39.9509,48.8994,78.71,43,0,0.56,AZ,1631772779,Tabassum Hotel
2,241,Artyom,40.4723,50.3330,78.94,57,0,1.72,AZ,1631772783,F marine resort
3,246,Saint-Pierre,-21.3393,55.4781,78.44,64,0,5.75,RE,1631772697,Le Battant Des Lames


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))